# Preparação do Ambiente


In [22]:
pip install tensorflow keras


In [28]:
pip install tensorflow


SyntaxError: invalid syntax (<ipython-input-28-a4235d8e22c9>, line 1)

In [29]:
pip install tensorflow scikit-learn


In [52]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_auc_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from scipy.stats import uniform

# Função para criar o modelo
def create_model(learning_rate=0.001, neurons=16, activation='relu', dropout_rate=0.2):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1],)))  #  Input shape para a primeira camada
    model.add(Dense(neurons, activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Função para treinar e avaliar o modelo
def build_and_evaluate_model(learning_rate, neurons, activation, dropout_rate, batch_size, epochs):
    model = create_model(learning_rate=learning_rate, neurons=neurons, activation=activation, dropout_rate=dropout_rate)
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0, validation_split=0.2)
    y_pred_proba = model.predict(X_test).ravel()  # Probabilidades preditas
    y_pred = (y_pred_proba > 0.5).astype("int32")  # Converção para classes binárias

    metrics = {}
    if len(np.unique(y_test)) > 1:
        metrics['accuracy'] = accuracy_score(y_test, y_pred)
        metrics['recall'] = recall_score(y_test, y_pred, zero_division=1)
        metrics['f1_score'] = f1_score(y_test, y_pred)
        metrics['roc_auc'] = roc_auc_score(y_test, y_pred_proba)
    else:
        metrics['accuracy'] = accuracy_score(y_test, y_pred)
        metrics['recall'] = 0
        metrics['f1_score'] = 0
        metrics['roc_auc'] = 0

    return metrics

# Carregamento dados
url = "/content/Creditos - Sheet1.csv"  # Substitua pelo link de conteúdo real
df = pd.read_csv(url)

# Pré-processamento
X = df.drop(columns=['Class'])  # Assumindo que 'Class' é a coluna de rótulo
y = df['Class']

# Divisão de dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Normalização as features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Parâmetros para Grid Search
param_grid = {
    'learning_rate': [0.001, 0.01],
    'neurons': [8, 16, 32],
    'activation': ['relu', 'tanh'],
    'dropout_rate': [0.2, 0.4],
    'batch_size': [16, 32, 64],
    'epochs': [10, 20]
}

# Grid Search
def grid_search(params):
    best_score = -1
    best_params = None
    for param_combination in pd.MultiIndex.from_product(params.values()).to_frame(index=False).itertuples(index=False):
        current_params = dict(zip(params.keys(), param_combination))
        metrics = build_and_evaluate_model(**current_params)
        if metrics['roc_auc'] > best_score:
            best_score = metrics['roc_auc']
            best_params = current_params
    return best_params, best_score

best_params_grid, best_score_grid = grid_search(param_grid)
print(f'Best Parameters from Grid Search: {best_params_grid}')
print(f'Best Grid Search ROC-AUC: {best_score_grid:.4f}')

# Random Search
param_distributions = {
    'learning_rate': uniform(0.001, 0.01),
    'neurons': [8, 16, 32],
    'activation': ['relu', 'tanh'],
    'dropout_rate': uniform(0.2, 0.4),
    'batch_size': [16, 32, 64],
    'epochs': [10, 20]
}

def random_search(params, n_iter=10):
    best_score = -1
    best_params = None
    for _ in range(n_iter):
        current_params = {}
        for key, value in params.items():
            if hasattr(value, 'rvs'):  # Verifica se value tem o método rvs
                current_params[key] = value.rvs()
            else:
                current_params[key] = np.random.choice(value)
        metrics = build_and_evaluate_model(**current_params)
        if metrics['roc_auc'] > best_score:
            best_score = metrics['roc_auc']
            best_params = current_params
    return best_params, best_score

best_params_random, best_score_random = random_search(param_distributions)
print(f'Best Parameters from Random Search: {best_params_random}')
print(f'Best Random Search ROC-AUC: {best_score_random:.4f}')

# Comparação e análise dos resultados
print("\nComparison of Models:")
print(f"Grid Search ROC-AUC: {best_score_grid:.4f}")
print(f"Random Search ROC-AUC: {best_score_random:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━

(10, 34.5, 3240, 0)

# Carregamento dos Dados